# Creating an intelligent and advanced RAG for movie recommendation
**A continuation to the previous project: [here](https://github.com/SajalPaudyal/Implementing_basic_RAG_for_movie_recommendation/blob/main/Information_Retrieval/a_movie_reccomender.ipynb)**

In [ ]:
import os
import faiss
import torch
import chromadb
import numpy as np
from dotenv import load_dotenv
from huggingface_hub import login
from langchain_groq import ChatGroq
from elasticsearch import Elasticsearch
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BertTokenizer, BertForSequenceClassification

/home/sajalpaudyal/Documents/Programming_Works/advanced_rag/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
load_dotenv()

hf_token = os.getenv("HF_TOKEN")
elastic_api = os.getenv("ELASTIC_API_KEY")
elastic_cloud_id = os.getenv("ELASTIC_CLOUD_ID")
groq_api_key = os.getenv("GROQ_API_KEY")

login = login(hf_token)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
sentence_transformer = SentenceTransformer("all-MiniLM-L6-v2")


In [4]:
def get_llm():
    model = ChatGroq(
        model="qwen/qwen3-32b",
        temperature = 1.0,
        max_tokens=1024,
        timeout=None,
        max_retries=2
    )
    
    return model

In [5]:
embeddings = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

In [6]:

es = Elasticsearch(cloud_id=elastic_cloud_id, api_key=elastic_api)
es.ping()


True

In [7]:
index_body = {
    "mappings": {
        "properties": {
            "content": {"type": "text"}
        }
    }
}

index_name = 'movies'
if not es.indices.exists(index=index_name):
    es.indices.create(index=index_name, body=index_body)
    print(f"Index '{index_name}' created.")
else:
    print(f"Index '{index_name}' already exists.")


Index 'movies' already exists.


In [8]:
rerank_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

rerank_model = BertForSequenceClassification.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
import getpass

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [ ]:
get_groq_model = get_llm()

In [ ]:
def advanced_query_transformation(query):
    expanded_query = query + " OR related_term"
    return expanded_query

In [ ]:
def advanced_query_routing(query):
    if "specific_keyword" in query:
        return "textual"
    else:
        return "vector"

In [ ]:
# Fusion Retrieval Function
def fusion_retrieval(query, top_k=5):
    
    query_embedding = sentence_transformer.encode(query).tolist()
    vector_results = collection.query(query_embeddings=[query_embedding], n_results=min(top_k, len(documents)))

    # Textual retrieval using Elasticsearch
    es_body = {
        "size": top_k,  # Move size into body
        "query": {
            "match": {
                "content": query
            }
        }
    }
    es_results = es.search(index=index_name, body=es_body)
    es_documents = [hit["_source"]["content"] for hit in es_results['hits']['hits']]
    
    combined_results = vector_results['documents'][0] + es_documents

    return combined_results

In [ ]:
import torch.nn.functional as F

def rerank_documents(query, documents):
    """
    Reranks the retrieved documents based on their relevance to the query using a pre-trained
    BERT model.

    Args:
        query (str): The user's query.
        documents (list): A list of documents retrieved from the search.

    Returns:
        list: A list of reranked documents, sorted by relevance.
    """
    inputs = [rerank_tokenizer.encode_plus(query, doc, return_tensors='pt', truncation=True, padding=True) for doc in documents]

    scores = []
    for input in inputs:
        outputs = rerank_model(**input)
        logits = outputs.logits
        probabilities = F.softmax(logits, dim=1)
        positive_class_probability = probabilities[:, 1].item()  # Assuming the second element represents the positive class
        scores.append(positive_class_probability)

    ranked_docs = sorted(zip(documents, scores), key=lambda x: x[1], reverse=True)
    return [doc for doc, score in ranked_docs]


In [ ]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Device set to use cpu


In [16]:
def select_and_compress_context(documents):
    """
    Summarizes the content of the retrieved documents to create a compressed context.

    Args:
        documents (list): A list of documents to summarize.

    Returns:
        list: A list of summarized texts for each document.
    """
    summarized_context = []
    for doc in documents:
        input_length = len(doc.split())  # Calculate input length based on word count
        max_length = min(100, input_length)  # Set max_length to input_length if smaller than 100
        summary = summarizer(doc, max_length=max_length, min_length=5, do_sample=False)[0]['summary_text']
        summarized_context.append(summary)
    return summarized_context

In [ ]:
# Answer Generation Function
def generate_answer(query, chunks, llm):
    """
    Generates an answer based on the input query and context chunks using a language model.

    Args:
        query (str): The user's query.
        chunks (list): A list of context chunks to inform the answer.
        llm (ChatGroq): An instance of the ChatGroq language model.

    Returns:
        str: The generated answer.
    """
    context = "\n\n".join(chunks)

    prompt = f"""[INST]
Instruction: You're an expert in movie suggestions. Your task is to analyze carefully the context and come up with an exhaustive answer to the following question:
{query}

Here is the context to help you:

{context}

[/INST]"""

    response = llm.invoke(prompt)  

    generated_text = response.content

    return generated_text


In [ ]:
def advanced_rag_pipeline(query):
    """
    The main pipeline function for the Advanced Retrieval-Augmented Generation (RAG) system.
    It processes the query, retrieves relevant documents, reranks them, selects and compresses
    the context, and finally generates an answer.

    Args:
        query (str): The user's input query.

    Returns:
        str: The final generated answer.
    """
    transformed_query = advanced_query_transformation(query)
    retrieval_method = advanced_query_routing(transformed_query)

    retrieved_documents = fusion_retrieval(transformed_query)

    ranked_documents = rerank_documents(query, retrieved_documents)

    context = select_and_compress_context(ranked_documents)

    final_answer = generate_answer(query, context, get_groq_model)
    return final_answer

In [ ]:
client = chromadb.Client()

collection_name = "movies"

try:
    collection = client.create_collection(name=collection_name)
    print(f"Collection '{collection_name}' created successfully.")

    documents = [
        {"id": "1", "content": "The Shawshank Redemption is a great movie to watch on a rainy day."},
        {"id": "2", "content": "Forrest Gump is an uplifting film perfect for a rainy afternoon."}
    ]

    ids = [doc["id"] for doc in documents]
    contents = [doc["content"] for doc in documents]

    collection.add(ids=ids, documents=contents)
    print("Documents inserted successfully.")

except Exception as e:
    print(f"Collection '{collection_name}' already exists. No need to create it again.")
    collection = client.get_collection(name=collection_name)

except Exception as e:
    print(f"An error occurred: {e}")


Collection 'movies' created successfully.
Documents inserted successfully.


In [ ]:
query = "What are some good movies to watch on a rainy day?"

answer = advanced_rag_pipeline(query)

print(answer)

<think>
Okay, I need to come up with a list of good movies to watch on a rainy day based on the context provided. The context mentions "Forrest Gump is an uplifting film" and "The Shawshank Redemption is a great movie to..." but it's cut off. Let me start by thinking about what makes a movie good for rainy days.

Rainy days often make people want to stay inside, so maybe comfort movies that are feel-good, nostalgic, or have a warm atmosphere. The context mentions uplifting films and The Shawshank Redemption, which is a classic. Shawshank is more of a prison drama with hope, but it's a popular choice. So maybe include uplifting movies with emotional depth.

Forrest Gump is a good example, so other uplifting dramas might work. Also, maybe classics, comedies for a lighter mood, or dramas that evoke reflection. I should consider different genres but focus on films that are cozy or have a satisfying story that keeps you engaged all day.

The user might want a mix of genres for variety. Let 